In [57]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()

In [58]:
sc = spark.sparkContext  # 필요 시 RDD API를 위해 사용

In [59]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

In [60]:
lines = sc.textFile("file:///home/jovyan/work/learning_spark_data/restaurant_reviews.csv")

lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [61]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [62]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [63]:
parse('0,짜장면,중식,125,')

('중식', 125)

In [64]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[65] at RDD at PythonRDD.scala:53

In [65]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [66]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [67]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [68]:
average = reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

## Non used Persist 

In [69]:
# transformations를 수행할 RDD 생성
categoryReviews = filtered_lines.map(parse)
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [70]:
result1 = categoryReviews.take(10) # action을 곧바로 실행

In [71]:
result2 = categoryReviews.mapValues(lambda x : (x, 1)).collect()

In [72]:
# 카테고리별 값1 합계 계산 (첫 번째 연산)
result_sum = categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
result_sum

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

## Used Persist

In [73]:
#job x
# categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews = filtered_lines.map(parse).persist() 
categoryReviews

PythonRDD[82] at RDD at PythonRDD.scala:53

In [74]:
result3 = categoryReviews.take(10)
result3

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [75]:
result4 = categoryReviews.mapValues(lambda x : (x, 1)).collect()
result4

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [76]:
# 카테고리별 값1 합계 계산 (첫 번째 연산)
result_sum = categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()
result_sum

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [77]:
# (카테고리, (점수, 1)) 형태로 변환
rdd_with_count = categoryReviews.map(lambda x: (x[0], (x[1], 1)))
rdd_with_count.take(3)

[('중식', (125, 1)), ('중식', (235, 1)), ('분식', (32, 1))]

In [78]:
# reduceByKey로 (점수 총합, 개수 총합) 집계
rdd_sums = rdd_with_count.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
rdd_sums.take(3)

[('중식', (360, 2)), ('분식', (566, 2)), ('일식', (287, 3))]

In [79]:
# 평균 계산
rdd_avg = rdd_sums.mapValues(lambda x: round(x[0] / x[1], 2))

# 결과 출력
rdd_avg.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.67), ('아시안', 312.0), ('패스트푸드', 17.5)]

In [80]:
# 기존 저장 레벨 해제
categoryReviews.unpersist()

PythonRDD[82] at RDD at PythonRDD.scala:53

In [81]:
from pyspark import StorageLevel
categoryReviews.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[82] at RDD at PythonRDD.scala:53

In [86]:
# 구조 확인하기
categoryReviews.take(3)

[('중식', 125), ('중식', 235), ('분식', 32)]

In [87]:
result11 = categoryReviews.filter(lambda x: x[1] > 100).count()
result12 = categoryReviews.filter(lambda x: x[1] <= 100).count()

# flatMap()

In [88]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

In [89]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

### 텍스트를 단어로 쪼개서 flatMap()

In [90]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [91]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[108] at readRDDFromFile at PythonRDD.scala:289

In [92]:
#job 추가 x
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

##  집합 Transformation

In [93]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

In [94]:
num1.intersection(num2).collect()

[4, 5]

###  합집합 구하기 - union

In [100]:
num_union = num1.union(num2)
num_union

UnionRDD[133] at union at NativeMethodAccessorImpl.java:0

In [101]:
num_union.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

### 차집합 구하기 - subtract

In [102]:
num1.subtract(num2).collect()

[1, 2, 3]

## 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)

In [105]:
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.3).collect()

[2, 5, 5, 10]

In [107]:
# withReplacement : False -> 중복 X
num_union.sample(False, 0.7).collect()

[2, 3, 4, 5, 7, 8, 10]

In [108]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
num_union.sample(True, 0.5, seed=42).collect()

[4, 5, 5, 5, 7]

## Wide Transformations

In [109]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[146] at readRDDFromFile at PythonRDD.scala:289

In [117]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[175] at RDD at PythonRDD.scala:53

In [118]:
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f58d0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f5910>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f5990>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f1f3c5f5a90>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f5ad0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f5b10>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f1f3c1f5bd0>)]

In [120]:
for (k, v) in foodsGroup.collect():
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
마 ['마라탕']
떡 ['떡볶이']


## 내로우 narrow  트랜스포메이션

In [127]:
numbers = sc.parallelize([1, 2, 3, 4, 5])

# 1:1 변환 (Narrow Transformation)
sample_rdd = numbers.sample(False, 0.5)
sample_rdd.take(3)

[1, 2, 3]

[1, 2, 3, 4, 5]

## 5. aggregate 함수 사용 예제

In [129]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6], numSlices=3)
numbers.collect()

[1, 5, 3, 9, 2, 8, 4, 7, 6]

glom()은 각 파티션의 내용을 리스트로 감싸서 반환   
즉, 파티션 간의 이동이 없고, 내부 자료구조만 바꾸는 연산   
glom()은 RDD의 각 파티션을 배열 형태로 변환하는 narrow transformation

In [131]:
numbers.glom().collect()

[[1, 5, 3], [9, 2, 8], [4, 7, 6]]

In [132]:
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6])

In [133]:
# 평균 계산하기
sum_count = numbers.aggregate(
    (0, 0),  # 초기값 (합계, 개수)
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # 각 파티션 내 연산
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # 파티션 간 연산
)
sum_count

(45, 9)

In [134]:
average = sum_count[0] / sum_count[1]
print("Average:", average)  # 5.0

Average: 5.0


In [138]:
# aggregate 연습
data = ["hello", "world", "spark"]
rdd = sc.parallelize(data)

def seq_op(acc, value):
    text, idx = acc
    return (f"{text} {idx}:{value}".strip(), idx + 1)

def comb_op(acc1, acc2):
    # 줄 번호 이어붙이기: 앞쪽 text 유지 + 뒤쪽 text 뒤에 붙이기
    text1, idx1 = acc1
    text2, idx2 = acc2
    # 줄 번호 충돌 피하려면 idx 조정 필요 (복잡하므로 단순히 붙임)
    return (f"{text1} {text2}".strip(), idx1 + idx2)

In [139]:
zero = ("", 1)

result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:hello 1:world 2:spark


In [140]:
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])
result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:spark 2:hadoop 1:ai 2:python 3:sql


In [141]:
sc.stop()
spark.stop()